# Data as Documents Solution

In [ ]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import wurst as w

In [ ]:
bd.projects.set_current("Bicycle example")

In [ ]:
data = w.extract_brightway2_databases(["🚲"])

In [ ]:
def get_co2(obj):
    return obj["type"] == bd.labels.biosphere_node_default and obj["name"] == "Carbon Dioxide"


co2 = w.get_one(data, get_co2)

In [ ]:
particles = {
    "database": "🚲",
    "type": bd.labels.biosphere_node_default,
    "code": "particles",
    "name": "particles",
    "unit": "kilogram",
    "categories": ("air",),
    "exchanges": [],
}
electricity = {
    "database": "🚲",
    "type": bd.labels.product_node_default,
    "code": "electricity",
    "name": "electricity",
    "unit": "kilowatt hour",
    "location": "SE",
    "exchanges": [],
}
electricity_generation = {
    "database": "🚲",
    "type": bd.labels.process_node_default,
    "code": "electricity generation",
    "name": "electricity generation",
    "location": "SE",
    #Add the following exchanges to the electricity generation:
    #
    #    Production of electricity
    #    Emission of carbon dioxide
    "exchanges": [{
        "name": "electricity",
        "unit": "kilowatt hour",
        "location": "SE",
        "amount": 1,
        "type": bd.labels.production_edge_default
    }, {
        "name": co2["name"],
        "unit": co2["unit"],
        "categories": co2["categories"],
        "amount": 0.1,
        "type": bd.labels.biosphere_edge_default    
    }]
}

In [ ]:
data.append(particles)
data.append(electricity)
data.append(electricity_generation)
# Could also do data.extend([particles, electricity, electricity_generation])

Add an exchange to emit particulates from the carbon fibre production

In [ ]:
def process_only_carbon_fibre(obj):
    return (
        obj["type"] == bd.labels.process_node_default 
        and obj["name"] == 'carbon fibre production'
    )


cf_production = w.get_one(data, process_only_carbon_fibre)
cf_production["exchanges"].append({
    "name": "particles",
    "unit": "kilogram",
    "categories": ("air",),
    "amount": 1e-4,
    "type": bd.labels.biosphere_edge_default        
})

Add an exchange which consumes electricity in the bicycle manufacture

In [ ]:
def process_only_bike_manufacture(obj):
    return (
        obj["type"] == bd.labels.process_node_default 
        and obj["name"] == 'bike production'
    )


bike_production = w.get_one(data, process_only_bike_manufacture)
bike_production["exchanges"].append({
    "name": "electricity",
    "unit": "kilowatt hour",
    "location": "SE",
    "amount": 0.15,
    "type": bd.labels.consumption_edge_default
})

In [ ]:
from wurst.brightway.write_database import link_internal_products_processes
count = link_internal_products_processes(data)

In [ ]:
if "🚲 🚬 😔" in bd.databases:
    del bd.databases["🚲 🚬 😔"]

w.write_brightway2_database(
    data, 
    name="🚲 🚬 😔", 
    products_and_processes=True
)

In [ ]:
bike = bd.get_node(name="bicycle", database="🚲 🚬 😔")
lca = bc.LCA({bike: 1})
lca.lci()
lca.supply_array

Check to make sure we have correct values in biosphere matrix:

In [ ]:
from math import isclose

co2 = bd.get_node(name="Carbon Dioxide", database="🚲 🚬 😔")
elec_prod = bd.get_node(name="electricity generation", database="🚲 🚬 😔")

assert isclose(
    lca.biosphere_matrix[
        lca.dicts.biosphere[co2.id],
        lca.dicts.activity[elec_prod.id]
    ],
    0.1,
    abs_tol=1e-6
), "Incorrect value in biosphere matrix"

In [ ]:
particles = bd.get_node(name="particles", database="🚲 🚬 😔")
cf_prod = bd.get_node(name="carbon fibre production", database="🚲 🚬 😔")

assert isclose(
    lca.biosphere_matrix[
        lca.dicts.biosphere[particles.id],
        lca.dicts.activity[cf_prod.id]
    ],
    1e-4,
    abs_tol=1e-6
), "Incorrect value in biosphere matrix"